In [80]:
import pandas as pd
import yfinance as yf
import talib as talib
import matplotlib.pyplot as plt
from backtesting import Strategy, Backtest

In [ ]:
"""
    Market: S&P 500
    Entry: Price above 500-day SMA & 10-period RSI below 30 
    Exit: close price below 500-day SMA
"""

In [2]:
pd.options.mode.chained_assignment = None

In [61]:
# create the DataFrame using YahooFinance
df = yf.download('^GSPC', start='1995-01-01')

[*********************100%***********************]  1 of 1 completed


In [78]:
class MA_RSI(Strategy):
    n1 = 500
    n2 = 10
    
    def init(self):
        self.sma = self.I(talib.SMA, self.data.Close, self.n1)
        self.rsi = self.I(talib.RSI, self.data.Close, self.n2)
        
    def next(self):
        if self.data.Close > self.sma and self.rsi < 25:
            self.buy()
        elif self.data.Close < self.sma:
            self.position.close()
        
        
        
                

In [79]:
bt = Backtest(df, MA_RSI, cash=10_000, commission=0.04, exclusive_orders=False)
stats = bt.run()
print(stats)
bt.plot(plot_volume=False)


Start                     1995-01-03 00:00:00
End                       2022-03-18 00:00:00
Duration                   9936 days 00:00:00
Exposure Time [%]                   48.248687
Equity Final [$]                 25694.981582
Equity Peak [$]                  32448.133164
Return [%]                         156.949816
Buy & Hold Return [%]              858.140764
Return (Ann.) [%]                    3.531673
Volatility (Ann.) [%]               10.722549
Sharpe Ratio                         0.329369
Sortino Ratio                        0.466843
Calmar Ratio                         0.126839
Max. Drawdown [%]                  -27.843814
Avg. Drawdown [%]                   -2.070471
Max. Drawdown Duration     2264 days 00:00:00
Avg. Drawdown Duration       56 days 00:00:00
# Trades                                    7
Win Rate [%]                        57.142857
Best Trade [%]                      70.459622
Worst Trade [%]                    -14.120453
Avg. Trade [%]                    

Row(id='22069', ...)

/usr/bin/sensible-browser: 8: /mnt/c/Program: not found
[10377:10414:0318/083948.503555:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[10377:10414:0318/083948.503675:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[10377:10413:0318/083948.508311:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[10377:10413:0318/083948.508440:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")


Opening in existing browser session.


[10421:10421:0100/000000.946782:ERROR:sandbox_linux.cc(377)] InitializeSandbox() called with multiple threads in process gpu-process.
